# **Uso de filtros y contornos para la localización de placas en imágenes de coches**
## **Objetivos**
A lo largo de esta actividad se pretende conocer y aplicar los conceptos básicos para el procesamiento de imágenes.
- Aplicar operaciones de suavizado.
- Utilizar operadores espaciales.
- Utilizar operadores morfológicos.
## **Descripción de la actividad**
El problema de reconocimiento de placas (también conocido como reconocimiento de matrículas) consiste en identificar y leer las placas de matrícula de los vehículos en imágenes o vídeos. Este problema es importante en una variedad de aplicaciones, como la seguridad de la carretera, la vigilancia y el seguimiento de vehículos.  
El proceso de reconocimiento de placas generalmente incluye varios pasos, como la detección de la placa en la imagen, la segmentación de los caracteres individuales en la placa y la identificación y extracción del texto en los caracteres. A menudo, se utilizan técnicas de procesamiento de imágenes, como la detección de bordes y la transformada de Hough, para detectar la placa. Luego, se pueden utilizar técnicas de aprendizaje automático, como el reconocimiento de caracteres con OCR, para identificar el texto en la placa.  
En esta actividad utilizaremos funciones básicas para llevar a cabo este proceso, el cual nos arroja como resultado la ubicación de regiones candidatas a ser placas. El código aquí presentado puede ser optimizado con otras técnicas; solamente se muestra una aproximación para conocer las diferentes funciones de procesamiento de imágenes que nos ayudarán a realizar esta tarea.
## **Preguntas**

### **1.	En el siguiente código vamos a leer la imagen de un coche tanto a color como en niveles de gris, responde correctamente cuáles son los valores correctos que arroja alto 1 y ancho 1 y cuántos canales tiene:**
**A. 248x400, 1 canal.**  
**<ins>B. 248 x 400, 3 canales.</ins>✅**  
**C. 320 x 460, 3 canales.**  
**D. 248 x 420, 1 canal.**

In [ ]:
import cv2 as cv

img01=cv.imread('Imagenes/cars96.png')
img02=cv.imread('Imagenes/cars96.png',cv.IMREAD_GRAYSCALE)
alto1, ancho1, canales1  = img01.shape
alto2, ancho2  = img02.shape

print("Alto1: ", alto1, "ancho1: ", ancho1, "canales: ", canales1)
print("Alto2: ", alto2, "ancho2: ", ancho2)

cv.imshow("img01",img01)
cv.imshow("img02",img02)
cv.waitKey(0)
cv.destroyAllWindows()

### **2.	El kernel aplicado a la imagen cars3.png es un filtro espacial____, y el resultado es la imagen:**
** Pasa **  
![Pasa ](ResultadoImagenes/kernel.png)

In [ ]:
import cv2 as cv
import numpy as np

img_gris=cv.imread('Imagenes/cars3.png',cv.IMREAD_GRAYSCALE)

kernel = np.array([[-1,-1,-1],
                   [-1, 8,-1],
                   [-1,-1,-1]])

img_ker = cv.filter2D(src=img_gris, ddepth=-1, kernel=kernel)

cv.imshow('Imagen con kernel', img_ker)
cv.waitKey(0)
cv.destroyAllWindows()

### **3.	El kernel aplicado a la imagen es un filtro ________ y se aplica principalmente para _________________. La imagen resultante es:**
** **  
![](ResultadoImagenes\3-GaussianBlur.png)

In [ ]:
import cv2 as cv

img_gris = cv.imread('Imagenes/cars296.png',cv.IMREAD_GRAYSCALE)
kernel = cv.GaussianBlur(img_gris, (9, 9), 0)

cv.imshow("Imagen",img_gris)
cv.imshow("Imagen con filtro", kernel)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('ResultadoImagenes/3-GaussianBlur.png',kernel)

### **4.	El objetivo de la ecualización de imágenes es ________. El resultado al aplicar ecualización a la imagen coche11.jpg es:**
**<ins>B. El objetivo de la ecualización es mejorar el contraste en una imagen, la imagen resultante es:</ins>✅** 
   
![](ResultadoImagenes/ecualizacion.png)

In [ ]:
# Cargar la imagen en color
import cv2 as cv
img = cv.imread('Imagenes/coche11.jpeg')

# Separa los canales de color de la imagen
b, g, r = cv.split(img)

# Ecualiza el histograma de cada canal de color
b_eq = cv.equalizeHist(b)
g_eq = cv.equalizeHist(g)
r_eq = cv.equalizeHist(r)

# Combina los canales de nuevo en una sola imagen 
img_eq = cv.merge((b_eq, g_eq, r_eq))
cv.imshow('Equalizada', img_eq)
cv.waitKey(0)
cv.destroyAllWindows
cv.imwrite('ResultadoImagenes/4-Ecualizacion.png',img_eq)

### **5.	Para llevar a cabo el reconocimiento de placas es necesario antes localizarlas, existen una gran cantidad de condiciones que afectan una imagen como puede ser la iluminación, distancia y rotaciones, entre otros. Revisa las diferentes imágenes de coches que acompañan la actividad para que identifiques estos cambios. Para lograr el objetivo, hemos definido una metodología para encontrar placas candidatas en una imagen mediante técnicas tradicionales de procesamiento de imágenes, la metodología sugerida es la siguiente:**

Paso 1. Leer imagen a color y transformar la imagen de color a gris.
```python
1  import cv2 as cv
2  import numpy as np
3  
4  
5  img_color = cv.imread('Imagenes/coche16.jpeg')
6  img_gris = cv.imread('Imagenes/coche16.jpeg', cv.IMREAD_GRAYSCALE)
```
Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
```python
8  blur = cv.GaussianBlur(img_gris, (11, 11), 0)
9  cv.imshow("Gausiano",blur)
```
Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
```python
11  clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
12  equ01 = clahe.apply(blur)
13  cv.imshow("Ajuste de contraste", equ01)
```
Paso 4. Binarizar la imagen (Otsu).
```python
15  otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
16  cv.imshow("Imagen binarizada", otsu01)
```
Paso 5. Obtener bordes (CANNY).
```python
18  edges = cv.Canny(otsu01, 150, 350)
19  cv.imshow("Canny", edges)
```
Paso 6. Guardar bordes.
```python
21  contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
22  long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
23                                               and (cv.arcLength(cnt, True) < 700 ))]
```
Paso 7. Dibujar placas candidatas.
```python
25  alto, ancho, canales = img_color.shape
26  for lonc in (long_contours):
27      r = [x for [[x,y]] in lonc]
28      s = [y for [[x,y]] in lonc]
29      prop = (max(s)-min(s))-(max(r)-min(r))
30  
31      if ((max(s)>round(alto/2)) and prop<0):
32          cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
```
Paso 8. Cerrar ventanas y terminar.
```python
34  cv.imshow("Imagen resultante", img_color)
35  cv.waitKey()
36  cv.destroyAllWindows()
```

**Al ejecutar el código para la imagen coche16.jpeg, ¿cuál es la salida para los pasos 2, 3 , 4 y 5? Relaciona las columnas.**

| Imagen  |  Paso  |
|---------|--------|
| Imagen 1 (Imagen binarizada) |       |
| ![Imagen binarizada paso 4](ResultadoImagenes/5.4-Imagen_binarizada.png) | Paso 4 |
| Imagen 2 (Imagen con filtro canny)|       |
| ![Imagen canny paso 5](ResultadoImagenes/5.5-Canny.png) | Paso 5 |
| Imagen 3 (Imagen con ajuste de contraste) |       |
| ![Imagen con ajuste de contraste paso 3](ResultadoImagenes/5.3-Mejora_de_contraste.png) | Paso 3 |
| Imagen 4 (Imagen con filtro gaussiano) |       |
| ![Imagen gaussiano paso 2](ResultadoImagenes/5.2-Gaussiano.png) | Paso 2 |

#### **Desmostración**

In [ ]:
# Paso 1. Leer imagen a color y transformar la imagen de color a gris.
import cv2 as cv
import numpy as np

img_color = cv.imread('Imagenes/coche16.jpeg')
img_gris = cv.imread('Imagenes/coche16.jpeg', cv.IMREAD_GRAYSCALE)
# Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
blur = cv.GaussianBlur(img_gris, (11, 11), 0)
cv.imshow("Gausiano",blur)
# Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
equ01 = clahe.apply(blur)
cv.imshow("Ajuste de contraste", equ01)
# Paso 4. Binarizar la imagen (Otsu).
otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
cv.imshow("Imagen binarizada", otsu01)
# Paso 5. Obtener bordes (CANNY).
edges = cv.Canny(otsu01, 150, 350)
cv.imshow("Canny", edges)
# Paso 6. Guardar bordes.
contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
                                             and (cv.arcLength(cnt, True) < 700 ))]
# Paso 7. Dibujar placas candidatas.
alto, ancho, canales = img_color.shape
for lonc in (long_contours):
    r = [x for [[x,y]] in lonc]
    s = [y for [[x,y]] in lonc]
    prop = (max(s)-min(s))-(max(r)-min(r))

    if ((max(s)>round(alto/2)) and prop<0):
        cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
#Paso 8. Cerrar ventanas y terminar.
cv.imshow("Imagen resultante", img_color)
cv.waitKey()
cv.destroyAllWindows()

##### **Paso 2 Imagen con filtro gaussiano**
![Paso2 Filtro gaussiano](ResultadoImagenes/5.2-Gaussiano.png)
##### **Paso 3 Imagen con mejora de contraste**
![Paso3 Mejora de contraste](ResultadoImagenes/5.3-Mejora_de_contraste.png)
##### **Paso 4 Imagen binarizada**
![Paso4 Imagen binarizada](ResultadoImagenes/5.4-Imagen_binarizada.png)
##### **Paso 5 Imagen con filtro canny**
![Paso5 Filtro canny](ResultadoImagenes/5.5-Canny.png)
##### **Paso 8 Imagen resultante**
![Paso8 Imagen resultante](ResultadoImagenes/5.8-Imagen_resultante.png)

### **6.	Utilizando el siguiente algoritmo, si eliminamos el paso 2 de filtro gaussiano y lo aplicamos a la imagen coche14.jpeg, el resultado es:**
**A. No detecta la placa.**  
**<ins>B. Detecta más de cinco candidatos a placa.</ins>✅**   
**C. Detecta exclusivamente la placa.**  
**D. Detecta tres candidatos de placa.**  


Paso 1. Leer imagen a color y transformar la imagen de color a gris.
```python
1  import cv2 as cv
2  import numpy as np
3  
4  
5  img_color = cv.imread('Imagenes/coche16.jpeg')
6  img_gris = cv.imread('Imagenes/coche16.jpeg', cv.IMREAD_GRAYSCALE)
```
Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
```python
8  blur = cv.GaussianBlur(img_gris, (11, 11), 0)
9  cv.imshow("Gausiano",blur)
```
Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
```python
11  clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
12  equ01 = clahe.apply(blur)
13  cv.imshow("Ajuste de contraste", equ01)
```
Paso 4. Binarizar la imagen (Otsu).
```python
15  otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
16  cv.imshow("Imagen binarizada", otsu01)
```
Paso 5. Obtener bordes (CANNY).
```python
18  edges = cv.Canny(otsu01, 150, 350)
19  cv.imshow("Canny", edges)
```
Paso 6. Guardar bordes.
```python
21  contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
22  long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
23                                               and (cv.arcLength(cnt, True) < 700 ))]
```
Paso 7. Dibujar placas candidatas.
```python
25  alto, ancho, canales = img_color.shape
26  for lonc in (long_contours):
27      r = [x for [[x,y]] in lonc]
28      s = [y for [[x,y]] in lonc]
29      prop = (max(s)-min(s))-(max(r)-min(r))
30  
31      if ((max(s)>round(alto/2)) and prop<0):
32          cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
```
Paso 8. Cerrar ventanas y terminar.
```python
34  cv.imshow("Imagen resultante", img_color)
35  cv.waitKey()
36  cv.destroyAllWindows()
```

#### **Desmostracion:**

In [ ]:
# Paso 1. Leer imagen a color y transformar la imagen de color a gris.
import cv2 as cv
import numpy as np

img_color = cv.imread('Imagenes/coche14.jpeg')
img_gris = cv.imread('Imagenes/coche14.jpeg', cv.IMREAD_GRAYSCALE)
# Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
"""blur = cv.GaussianBlur(img_gris, (11, 11), 0)
cv.imshow("Gausiano",blur)"""
# Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
equ01 = clahe.apply(img_gris)
cv.imshow("Ajuste de contraste", equ01)
# Paso 4. Binarizar la imagen (Otsu).
otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
cv.imshow("Imagen binarizada", otsu01)
# Paso 5. Obtener bordes (CANNY).
edges = cv.Canny(otsu01, 150, 350)
cv.imshow("Canny", edges)
# Paso 6. Guardar bordes.
contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
                                             and (cv.arcLength(cnt, True) < 700 ))]
# Paso 7. Dibujar placas candidatas.
alto, ancho, canales = img_color.shape
for lonc in (long_contours):
    r = [x for [[x,y]] in lonc]
    s = [y for [[x,y]] in lonc]
    prop = (max(s)-min(s))-(max(r)-min(r))

    if ((max(s)>round(alto/2)) and prop<0):
        cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
#Paso 8. Cerrar ventanas y terminar.
cv.imshow("Imagen resultante", img_color)
cv.waitKey()
cv.destroyAllWindows()

##### **Imagen resultante, sin paso 2 de filtro gaussiano**
![Imagen sin filtro gaussiano](ResultadoImagenes/6.2-Imagen_res_Sin_Gaussiano.png)

## **7. Utilizando el siguiente algoritmo Si incluimos todos los pasos y lo aplicamos a la imagen coche14.jpeg, el resultado es:**
**A. Solo detecta la placa.**  
**B. Detecta tres candidatos a placa.**  
**<ins>C. Detecta dos candidatos a placa.</ins>✅**  
**D. No detecta ninguna placa.**  


Paso 1. Leer imagen a color y transformar la imagen de color a gris.
```python
1  import cv2 as cv
2  import numpy as np
3  
4  
5  img_color = cv.imread('coche16.jpeg')
6  img_gris = cv.imread('coche16.jpeg', cv.IMREAD_GRAYSCALE)
```
Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
```python
8  blur = cv.GaussianBlur(img_gris, (11, 11), 0)
9  cv.imshow("Gausiano",blur)
```
Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
```python
11  clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
12  equ01 = clahe.apply(blur)
13  cv.imshow("Ajuste de contraste", equ01)
```
Paso 4. Binarizar la imagen (Otsu).
```python
15  otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
16  cv.imshow("Imagen binarizada", otsu01)
```
Paso 5. Obtener bordes (CANNY).
```python
18  edges = cv.Canny(otsu01, 150, 350)
19  cv.imshow("Canny", edges)
```
Paso 6. Guardar bordes.
```python
21  contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
22  long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
23                                               and (cv.arcLength(cnt, True) < 700 ))]
```
Paso 7. Dibujar placas candidatas.
```python
25  alto, ancho, canales = img_color.shape
26  for lonc in (long_contours):
27      r = [x for [[x,y]] in lonc]
28      s = [y for [[x,y]] in lonc]
29      prop = (max(s)-min(s))-(max(r)-min(r))
30  
31      if ((max(s)>round(alto/2)) and prop<0):
32          cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
```
Paso 8. Cerrar ventanas y terminar.
```python
34  cv.imshow("Imagen resultante", img_color)
35  cv.waitKey()
36  cv.destroyAllWindows()
```

#### Demostración


In [ ]:
# Paso 1. Leer imagen a color y transformar la imagen de color a gris.
import cv2 as cv
import numpy as np

img_color = cv.imread('Imagenes/coche14.jpeg')
img_gris = cv.imread('Imagenes/coche14.jpeg', cv.IMREAD_GRAYSCALE)
# Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
blur = cv.GaussianBlur(img_gris, (11, 11), 0)
cv.imshow("Gausiano",blur)
# Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
equ01 = clahe.apply(blur)
cv.imshow("Ajuste de contraste", equ01)
# Paso 4. Binarizar la imagen (Otsu).
otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
cv.imshow("Imagen binarizada", otsu01)
# Paso 5. Obtener bordes (CANNY).
edges = cv.Canny(otsu01, 150, 350)
cv.imshow("Canny", edges)
# Paso 6. Guardar bordes.
contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
                                             and (cv.arcLength(cnt, True) < 700 ))]
# Paso 7. Dibujar placas candidatas.
alto, ancho, canales = img_color.shape
for lonc in (long_contours):
    r = [x for [[x,y]] in lonc]
    s = [y for [[x,y]] in lonc]
    prop = (max(s)-min(s))-(max(r)-min(r))

    if ((max(s)>round(alto/2)) and prop<0):
        cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
#Paso 8. Cerrar ventanas y terminar.
cv.imshow("Imagen resultante", img_color)
cv.waitKey()
cv.destroyAllWindows()

##### **Imagen resultante:**  
![7 Imagen resultante](ResultadoImagenes/7-Imagen_resultante.png)

### **8.	Utilizando el siguiente algoritmo, vamos a modificar unas líneas, vamos a aplicar un filtro gaussiano con un kernel de 3x3 en vez del de 11x11 (paso 2). Al aplicar el procedimiento de detección de placas, ¿cuántas placas detecta al aplicar el proceso a la imagen cars34.png? ¿Cómo se ve la imagen después de aplicar Canny?**
**A. No detecta ninguna placa.**  
**B. Detecta exclusivamente la placa.**  
**C. Detecta dos candidatos a placa.**  
**<ins>D. Detecta más de tres placas.</ins>✅**  

**Despues de aplicar Canny se ve:**  
  
  
Paso 1. Leer imagen a color y transformar la imagen de color a gris.
```python
1  import cv2 as cv
2  import numpy as np
3  
4  
5  img_color = cv.imread('Imagenes/coche14.jpeg')
6  img_gris = cv.imread('Imagenes/coche14.jpeg', cv.IMREAD_GRAYSCALE)
```
Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
```python
8  blur = cv.GaussianBlur(img_gris, (11, 11), 0)
9  cv.imshow("Gausiano",blur)
```
Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
```python
11  clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
12  equ01 = clahe.apply(blur)
13  cv.imshow("Ajuste de contraste", equ01)
```
Paso 4. Binarizar la imagen (Otsu).
```python
15  otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
16  cv.imshow("Imagen binarizada", otsu01)
```
Paso 5. Obtener bordes (CANNY).
```python
18  edges = cv.Canny(otsu01, 150, 350)
19  cv.imshow("Canny", edges)
```
Paso 6. Guardar bordes.
```python
21  contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
22  long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
23                                               and (cv.arcLength(cnt, True) < 700 ))]
```
Paso 7. Dibujar placas candidatas.
```python
25  alto, ancho, canales = img_color.shape
26  for lonc in (long_contours):
27      r = [x for [[x,y]] in lonc]
28      s = [y for [[x,y]] in lonc]
29      prop = (max(s)-min(s))-(max(r)-min(r))
30  
31      if ((max(s)>round(alto/2)) and prop<0):
32          cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
```
Paso 8. Cerrar ventanas y terminar.
```python
34  cv.imshow("Imagen resultante", img_color)
35  cv.waitKey()
36  cv.destroyAllWindows()
```

#### **Demostración**

In [ ]:
# Paso 1. Leer imagen a color y transformar la imagen de color a gris.
import cv2 as cv
import numpy as np

img_color = cv.imread('Imagenes/Cars34.png')
img_gris = cv.imread('Imagenes/Cars34.png', cv.IMREAD_GRAYSCALE)
# Paso 2. Suavizar la imagen (filtro gaussiano con kernel de 11x11).
blur = cv.GaussianBlur(img_gris, (3, 3), 0)
cv.imshow("Gausiano",blur)
# Paso 3. Mejora de contraste (ajuste local de contraste adaptativo).
clahe = cv.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
equ01 = clahe.apply(blur)
cv.imshow("Ajuste de contraste", equ01)
# Paso 4. Binarizar la imagen (Otsu).
otsu_threshold, otsu01 = cv.threshold( equ01, 0, 255, cv.THRESH_OTSU)
cv.imshow("Imagen binarizada", otsu01)
# Paso 5. Obtener bordes (CANNY).
edges = cv.Canny(otsu01, 150, 350)
cv.imshow("Canny", edges)
# Paso 6. Guardar bordes.
contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
long_contours = [cnt for cnt in contours if (cv.arcLength(cnt, True) > 200
                                             and (cv.arcLength(cnt, True) < 700 ))]
# Paso 7. Dibujar placas candidatas.
alto, ancho, canales = img_color.shape
for lonc in (long_contours):
    r = [x for [[x,y]] in lonc]
    s = [y for [[x,y]] in lonc]
    prop = (max(s)-min(s))-(max(r)-min(r))

    if ((max(s)>round(alto/2)) and prop<0):
        cv.rectangle(img_color,(min(r),min(s)),(max(r),max(s)),(0,255,0),3)
#Paso 8. Cerrar ventanas y terminar.
cv.imshow("Imagen resultante", img_color)
cv.waitKey()
cv.destroyAllWindows()

##### **Imagen "Canny"**
![8ImagenCanny](ResultadoImagenes/8-Canny.png)
##### **Imagen resultante**
![8ImagenResultante](ResultadoImagenes/8-Imagen_resultante.png)